In [1]:
from pathlib import Path
import os.path
import os
import numpy as np
import re
import pandas as pd
import shutil


# create network 
1. get_node  
2. get_edge
3. get_network_fixed
    1. calls get_edge 
    2. get_node
4. the code calling get_net_work_fixed
5. add time stamp


In [2]:
soft_path = 'D:/mm_thesis/the_files/Data/'
csv_path =  'D:/mm_thesis/Software_graphs/D_graph/'
softwareList = os.listdir(soft_path)
soft_files = [(os.path.join(soft_path,infile)+"/") for infile in softwareList]
csv_path_l = os.listdir(csv_path)

In [143]:
soft_files



['D:/mm_thesis/the_files/Data/Jena/']

### 1 get_node 

In [3]:
def get_node(node_line): 
#     print(node_line)    
    nodes = node_line.find("[") - 1 
    node_no = node_line[0:nodes]  
    n_end = node_line.find(",h")         # remove specail characters and skip the shape 
    node_name = node_line[nodes+2:n_end] 
    O_name = node_name 
    O_name = re.sub(r'(label=)','',O_name) 
    O_name = O_name.strip('"') 
    node_name =re.sub(r'(\\l)','',node_name) 
    node_name = re.sub(r'[\n \|| \\"]','',node_name) 
    node_name = re.sub(r'(label=)','',node_name) 
    #print(node_no, node_name) 
    args = " " 
    arg_st = node_name.find("<") 
    #print(node_name) 
    if node_name.rfind(".") != -1:   
        st_s = node_name.rfind(".") +1 

        if arg_st == -1:    # if node is a class   
            node_type = "class"
            class_c = node_name[st_s:]  
            class_p = node_name[0:st_s-1] 
        else: 
            node_type = 'method' #it's a method 
            #arg_st = node_name.find("<") 
            #print(arg_st) 
            #if arg_st != -1 : 

            arg_end = node_name.find(">")+1     
            #print(arg_end) 
            args = node_name[arg_st:arg_end] 
            args = args.strip('<') 
            args = args.strip('>') 

            st_s = node_name.rfind(".") +1 
            node_name = node_name[0: arg_st] 

            st_sm = node_name.rfind(".") 
            if st_sm != -1: 
                class_c = node_name[st_sm+1: arg_st] 
                class_p = node_name[0:st_sm] 
            else: 
                class_c = "" 
                class_p = "" 

               #print(node_name) 
               #print(type(args), 'from func') 
           #        args = np.array(args.split(',')) 
               
               #print(\"class_method\",args) 
                
    else : 
       #check if it has the colon identifier  
       #if arg_st != -1: 
       #print(node_name) 
        st_s =  len(node_name) + 1   #node is a java method so class and package are empty  
        class_c = " " 
        class_p = "" 
        node_type = "method"          
        #arg_st = node_name.find(\"<\") 
        #print(arg_st) 
        if arg_st != -1: 

            arg_end = node_name.find(">")+1 

            #print(arg_end) 
            args = node_name[arg_st:arg_end] 
            args = args.strip('<') 
            args = args.strip('>') 
            #print(type(args), 'from func') 
            #args = np.array(args.split(',')) 
            node_name= node_name[0:arg_st] 
            if node_name.rfind("::") != -1:   
                st_s = node_name.rfind("::") +2 
                class_c = node_name[st_s:]  
                class_p = node_name[0:st_s-1] 
                #print(class_c, class_p) 
                #print(\"java\",args) 
               #print(len(args)) 

            
    return (node_no,O_name,node_name, class_c, class_p,node_type, args)


### 2 get_edge

In [4]:
def get_edge(edge_line): 
    nedge= edge_line.find("->") 
    nsource = nedge -1 
    Node_s = edge_line[0:nsource] 
    source_node = Node_s 
    edgel = edge_line.find("[") 
    edgen = edge_line.find("]") 
    ntargetend = edgel - 1 
    ntarget = nedge +3 
    Node_t = edge_line[ntarget:ntargetend] 
    target_node = Node_t 
    edge_attr = edge_line[edgel+1:edgen -1] 
    dir_st = edge_attr.find("=") 
    dir_end= edge_attr.find(",") 
    edge_dir = edge_attr[dir_st+1:dir_end] 
    edge_dir = edge_dir.strip('"') 

       
       #print(\"s --> t\", Node_s, Node_t) 
        
       ### the direction is for vizualization  so this is not correct 
   #     if edge_dir == \"back\": 
   #             #print('dir back switch nodes') 
   #         source_node  = Node_t 
   #         target_node = Node_s  
               #print(\"s --> t\", source_node, target_node) 
          
                   
    # get the edge attributes, color , style, label  
            
    edge_c= edge_attr.find("color") 
    edge_ce = edge_attr[edge_c:].find(",") 
    color_len= edge_c + edge_ce 
    edge_color = edge_attr[edge_c+ 6:color_len].strip('"') 

    edge_sl = edge_attr.find("style") 
    edge_se = edge_attr[edge_sl:].find(",") 
    style_len= edge_sl + edge_se 
    edge_style = edge_attr[edge_sl+ 6:style_len].strip('"') 

    edge_la = edge_attr.find("label") 
    edge_label = " " 

    if edge_la != -1: 
        edge_le = edge_attr[edge_la:].find(",") 
        label_len= edge_la + edge_le 
        edge_label = edge_attr[edge_la + 6:label_len] 
        edge_label = edge_label.strip(' ') 
        edge_label = edge_label.strip('"') 
#     print('get_edge')
#     print(source_node, target_node, edge_attr, edge_color, edge_style, edge_label)
#     print()
#     print(30*'=')

    return (source_node, target_node, edge_attr, edge_color, edge_style, edge_label)


## 3 get_net_work_fixed called get_edge and get_node

In [5]:
def get_network_fixed(dot_path, csv_graph,csv_node,ver): 
    fileList = os.listdir(dot_path) 
    dot_files = [os.path.join(dot_path,infile )for infile in fileList] 

    column_names = ['Nsource','Ntarget','s_o','t_o','edge_attr', 'e_color', 'e_style', 'e_label', 'source', 'target','ver','e_pack'] 
    #umn_names = ['source', 'target','edge_attr','Nsource','Ntarget'] 
    all_edges = pd.DataFrame(columns = column_names) 

    nodes_col = ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver'] 
    all_nodes = pd.DataFrame(columns = nodes_col) 
    all_nodes_e = pd.DataFrame(columns = nodes_col) 
    t = 1 

    for in_file in fileList: 
        #print(in_file) 
      
        
        ##print(\"iteration\",t) 
        t = t+1 
        f_name = (os.path.join(dot_path,in_file)) 
        ##print(f_name) 
        #f_name = (os.path.join(C_path,in_file)) 
        #f_name = (os.path.join(I_path,in_file)) 
        st = in_file.find(".dot") 
        csv_name = in_file[:st] 
        #print('csv_name',csv_name) 
        ##print(f_name) 

        edges_list = [] 
        Nodes_list = [] 

        lines = [ line.strip() for line in open(f_name)] 
        for i,line in enumerate(lines):   
            #print('line = ',  line)
            if i == 0:
#                 print('line = ',  line)
                e_pack_st = line.find('"') +1
                e_pack_en = line.rfind('"')
                e_pack = line[e_pack_st:e_pack_en]
#                 print('package ',e_pack)
            
            if line.find("->") != -1: #get the edges 
#                 print(line)
#                 print(30*'+')
#             #                
                #print(i, "  edge   ", line) 
    #             #print(\"first if\") 
    #             #print(50*'<>') 
                #print('++++++++++++++++++++++++++++++++++') 
#                 #print('edge', line) 
                l = line 
                src_st= line.rfind(" ->") 
                src_node= line[:src_st] 
#                 print('proper source:', src_node) 
                tar_st = line.rfind('>') 
                tar_node = line [src_st+4: line.find(' [')] 
#                 print('Target node : ',tar_node) 
#                 #print('+++++++++++++++++++++++++++++++++') 
    #             #print(\"nodes array    >\" , Nodes_arr) 
#                 print(line,'s_node===>', src_node, 't_node===>', tar_node) 
                source_node, target_node, edge_attr, edge_color, edge_style, edge_label= get_edge(line) 
#                 print('get_edge')
#                 print(source_node, target_node, edge_attr, edge_color, edge_style, edge_label)
               
                edges_list.append([source_node, target_node, edge_attr, edge_color, edge_style, edge_label,ver,e_pack]) 
# this way I get the nodes line no duplicates 
            if (line.find("->") == -1  and line.find("Node") !=-1 and i != 0):
#                 print(30*'^')
#                 print ('Node line   ','---',line)
#                 if i ==0 :
#                     print ('Node line   ','---',line)
#                     print(30*'^')
                node_no,O_name, node_name, node_class, node_pack,node_type,args = get_node(line) 
                if node_no.find('digraph') != -1:
                    print(line)
                    print(100*'==')
                    print(node_no,'----->',  in_file)
# #                 print('get_node')
#                 print(node_no,O_name, node_name)
                #print( 40*'#') 
                Nodes_list.append([node_no, O_name, node_name, node_type,node_class, node_pack,args,ver]) 
                
#         print('edge_list',edges_list)
#         print(100*'+')
  # Edges data frame      
        try: 
            edges = pd.DataFrame(edges_list) # get the edge list to a panda dataframe 
            edges.columns= ['Nsource','Ntarget','edge_attr','e_color', 'e_style', 'e_label','ver','e_pack'] 
#             print('file',in_file)
#             print(len(edges))
#             print(len(edges_list))
        except Exception: 
            print('err for', in_file )
            pass 
#         #print(edges.head()) 
#         print('nodes_list', len(Nodes_list),Nodes_list)
#         print(100*'-')
#         print(edges.columns)
#         print(edges.head(2))
#         print(100*'^')
        try: 
            if len(Nodes_list)!= 0:  
#                 print('in_try')
#                 print('nodes_list', len(Nodes_list),Nodes_list)
#                 print(100*'-')
                Nodes = pd.DataFrame(Nodes_list)   
        
                Nodes.columns= ['Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver'] 
#                 print(Nodes.head())
#                 print(Nodes.columns)


#                 #print(Nodes.head()) 
            #Nodes.drop_duplicates(subset = 'Node', keep='first',inplace=True) 
                edges= edges.assign(source =" ") 
                edges= edges.assign(target =" ") 
                edges= edges.assign(s_o =" ") 
                edges= edges.assign(t_o =" ") 

            # get the node names in the edged file 
                for i, row in edges.iterrows(): 
                    source = Nodes[Nodes['Node']==row['Nsource']]['N_name'].values[0] 
                    target = Nodes[Nodes['Node']==row['Ntarget']]['N_name'].values[0] 
                    src_old = Nodes[Nodes['Node']==row['Nsource']]['O_name'].values[0] 
                    trgt_old = Nodes[Nodes['Node']==row['Ntarget']]['O_name'].values[0]
                    
                    
                    edges.at[i,"source"]=source 
                    edges.at[i,"target"]=target 
                    edges.at[i,"s_o"]=src_old 
                    edges.at[i,"t_o"]=trgt_old

        except Exception:
            print("error node names")
            pass  

#         #print(edges.head())
#         print('Nodes in get network fixed', Nodes)
#         print(len(Nodes_list))
       
        try: 
            if len(Nodes) !=0: 
                edges= edges[['source','target','s_o','t_o','edge_attr','e_color', 'e_style',
                              'e_label','Nsource','Ntarget','ver','e_pack']] 
            #edges= edges[['source','target','edge_attr','Nsource','Ntarget']] 
                all_edges =  all_edges.append(edges,ignore_index = True) 
#                 print('all_edges', len(all_edges))
        except Exception:
            print('err in all_edegs')
            pass 

        #print(csv_graph+csv_name+'.csv') 
#         edges.to_csv(csv_graph+ csv_name+'_fixed_mm.csv', encoding='utf-8', index=False) 
#         Nodes.to_csv(csv_node+ csv_name+'_fixed_mm.csv', encoding = 'utf-8') 
        if len(Nodes)!= 0:   
            all_nodes = all_nodes.append(Nodes,ignore_index = True) 

            all_nodes_e = all_nodes.reset_index().drop_duplicates(subset = 'N_name',keep='first').set_index('index') 
            all_nodes_e = all_nodes_e.reset_index(drop=True) 
                #         # ADDING THE NODE ID COLUMN 
            all_nodes_e['Node_id'] = np.arange(len(all_nodes_e)) 
            all_nodes_e = all_nodes_e[['Node_id','Node','O_name','N_name','N_type', 'N_class', 'N_pack','args','ver']] 
    #     ############################################################################### 
#     print('nodes data_frame', all_nodes)
    all_edges= all_edges[['source','target','s_o', 't_o','edge_attr','e_color', 'e_style', 'e_label','Nsource','Ntarget','ver','e_pack']] 
    all_nodes_e.to_csv(csv_node+'Nodes_fixed.csv',encoding = 'utf-8',index = False)  
    print('file saves',csv_node+'Nodes_fixed.csv') 
    all_edges.to_csv(csv_graph+'all_fixed.csv',encoding='utf-8', index=False) 

# The main part calling get_network

In [48]:
soft_filesh = soft_files[2:3]
# print(soft_filesh)
# os.listdir(soft_filesh[0])



In [146]:
import itertools 
#for ver in soft_filesh:
for ver in soft_files:  
    print(ver)  

    soft_e = ver.rfind("/")  
    soft_n = ver[:soft_e]  
    soft_n1 = soft_n.rfind("/") +1  
    soft_name = ver[soft_n1:soft_e] +"/"  
#     print('soft name',soft_name)  
    #os.makedirs(csv_path, ver)   
    ver_list = os.listdir(ver)  
    #print(\"ver list:   \  ver_list)  
    #     print(\"VER list   :\  ver_list)  
    #     print(50*'#')  
    ver_files = [(os.path.join(ver,infile )+"/")for infile in ver_list]  
    for dot_graph in ver_files:  
        print("version",  dot_graph)  
        dot_csv = dot_graph.rfind('-')  
    #             print(dot_csv)  
        end_graph = dot_graph.rfind('/')  
        ver_no = dot_graph[dot_csv+1: end_graph]  

        print('version no:  ', ver_no)  
        dot_list = os.listdir(dot_graph)  

        graph_files = [(os.path.join(dot_graph,i )+"/")for i in dot_list]  


    #             print("dot graph:", dot_graph)  
        ver_loc = dot_graph.rfind("/")  

        #print(ver_loc)  
        ver_t = dot_graph[0:end_graph]  

        ver_loc_2 = ver_t.rfind("/")  
        ver_t2 = ver_t[ver_loc_2+1:]  
        ver_t2= ver_t2 +"/"  
        print( "VER_T2:" ,ver_t2)  
        print("DOT_list:", dot_list)  

        csv_path_files = [os.path.join(csv_path,"D_sys/",soft_name,ver_t2,i)+"_csv/" for i in dot_list]  
        print('CSV path    :',csv_path_files)  
        csv_nodes = [os.path.join(csv_path,"D_sys/",soft_name,ver_t2,i)+"_nodes/" for i in dot_list]  
        print('CSV nodes:', csv_nodes)  
        print(50*'#')  
        net_work = [os.makedirs(g,exist_ok=True) for g in csv_path_files]  


        net_nodes = [os.makedirs(n,exist_ok=True) for n in csv_nodes ]  
        if net_work ==[]:    
            net_work = csv_path_files 
        if net_nodes == None: 
            net_nodes = csv_nodes

#         print('DOT gratph : ', dot_graph) 
#         print('CSV path    :',csv_path_files) 
#         print('nodes path ', csv_nodes) 



            
        for (a, b, c) in zip(graph_files, csv_path_files, csv_nodes):  
            print("a,b,c,ver:", a,b,c,ver_no) 
            get_network_fixed(a,b,c,ver_no) 



    

D:/mm_thesis/the_files/Data/Jena/
version D:/mm_thesis/the_files/Data/Jena/jena-3.13.0/
version no:   3.13.0
VER_T2: jena-3.13.0/
DOT_list: ['CDot', 'CollDot', 'InhDot']
CSV path    : ['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/InhDot_csv/']
CSV nodes: ['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CDot_nodes/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CollDot_nodes/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/InhDot_nodes/']
##################################################
a,b,c,ver: D:/mm_thesis/the_files/Data/Jena/jena-3.13.0/CDot/ D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CDot_csv/ D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CDot_nodes/ 3.13.0
file saves D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.13.0/CDot_nodes/Nodes

In [24]:
dot_graph

'D:/mm_thesis/the_files/Data/Jena/jena-2.13.0/'

# getting nodes id in the edges file for source and target nodes
by getting the name of source & target from the edges file and matching them to the nodes file getting the Node_id 

# redo so you get the new files created 

In [6]:
csv_path = 'D:/mm_thesis/Software_graphs/D_graph/D_sys/'
csv_path_l = os.listdir(csv_path)
csv_files = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l]

In [7]:
# net_csv
csv_files

['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/']

In [8]:
for net in csv_files:
# for net in net_csv:    
   
    net_list = os.listdir(net)
    net_files = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
    
    
    #print(net_files)
    for net_graph in net_files:
        
        nn_list = os.listdir(net_graph)
        #print(nn_list)
        net_node_path = [(os.path.join(net_graph,i )+"/")for i in nn_list]
        #print(net_node_path)
        res = [[i , j] for i, j in zip(net_node_path[::2], net_node_path[1::2])] 
        for n in res:
            print(n[0])
            tt1 = os.listdir(n[0])      
            ttl =[i for i in tt1 if i.find('all') != -1]
            tt = ttl[0]
            file_net_work =(os.path.join(n[0]+tt))
            print("file edges", file_net_work)
            print(30 * '^')

            print(n)
            tn1 = os.listdir(n[1])
            print('tn1', tn1)
            #     print(len(tn1))
            #     print(n[1])
            tnl = [i for i in tn1 if i.find('Nodes_fixed') != -1]
            print('tnl',tnl)
            tn = tnl[0]
            print('tn', tn)
            file_node = os.path.join(n[1]+tn)
            print("file nodes",file_node,'\n')
            print(30 * '_')
            
            
            net_df = pd.read_csv(file_net_work)
            node_df = pd.read_csv(file_node)
        
            net_df= net_df.assign(s_id =" ")
            net_df= net_df.assign(t_id =" ")
            
#             print(net_df.head(2))
#             print(30*'^')
            for i, row in net_df.iterrows():
                
                
                s_i =node_df[node_df['N_name']==row['source']]['Node_id'].values[0]
                t_i =node_df[node_df['N_name']==row['target']]['Node_id'].values[0]
                net_df.at[i,"t_id"] = t_i
                net_df.at[i,"s_id"] = s_i
#                 row['t_id'] = t_i
#                 row['s_id'] = s_i
#                 print(row)
                
#             print(net_df.head(2))
#             print("net col BEFORE update", net_df.columns)
            
            net_df.rename(columns= {'source':'o_source', 'target' :'o_target','s_id': 'source', 't_id' : 'target'}, inplace = True)
            net_df = net_df[['source','target','o_source','o_target','s_o','t_o','edge_attr','e_color', 'e_style',
                              'e_label','Nsource','Ntarget','ver']]         
#             print("net col after update", net_df.columns)
#             print(net_df.head(2))
#             print(30* '*')
            net_df.to_csv(n[0]+"edges.csv", encoding='utf-8',index = False)

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_gr

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_gr

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_gr

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/CollDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/InhDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/InhDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/InhDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.10.0/InhDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/InhDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/InhDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/InhDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/InhDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/CollDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/CollDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/CollDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/CollDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/CollDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/InhDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/InhDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/InhDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.4.0/InhDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.9.0/InhDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.9.0/InhDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.9.0/InhDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.9.0/InhDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.9.0/InhDot_nodes/Nodes_fixed.csv 

______________________________
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-4.0.0/CDot_csv/
file edges D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-4.0.0/CDot_csv/all_fixed.csv
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-4.0.0/CDot_csv/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-4.0.0/CDot_nodes/']
tn1 ['Nodes_fixed.csv']
tnl ['Nodes_fixed.csv']
tn Nodes_fixed.csv
file nodes D:/mm_thesis/Software_graphs/D_grap

# adding the time stamp


In [9]:
csv_path = 'D:/mm_thesis/Software_graphs/D_graph/'
csv_path_l = os.listdir(csv_path)
csv_files1 = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l if os.path.isdir(os.path.join(csv_path,infile))]
csv_files = [i for i in csv_files1 if i.find("Time") == -1]
look_path = 'D:/mm_thesis/Software_graphs/lookup.csv'
# look_path = 'D:/mm_thesis/Software_graphs/hadoop.csv'


In [10]:
csv_files1

['D:/mm_thesis/Software_graphs/D_graph/D_sys/']

In [11]:
look_path = 'D:/mm_thesis/Software_graphs/lookup.csv'
Lookup_df = pd.read_csv(look_path)
Lookup_df.tail(50)

,tarfile,netfile,Timestamp,Time_created,yaml,stamp,d
198,hadoop-release-0.13.1.tar.gz,Hadoop_0131,1.245644e+09,2009-06-22-06:10,hadoop0131,1185235200,24-Jul-07
199,hadoop-release-0.14.0.tar.gz,Hadoop_0140,1.245644e+09,2009-06-22-06:10,hadoop0140,1187308800,17-Aug-07
200,hadoop-release-0.14.2.tar.gz,Hadoop_0142,1.245644e+09,2009-06-22-06:10,hadoop0142,1191628800,6-Oct-07
201,hadoop-release-0.14.3.tar.gz,Hadoop_0143,1.245644e+09,2009-06-22-06:10,hadoop0143,1192492800,16-Oct-07
202,hadoop-release-0.14.4.tar.gz,Hadoop_0144,1.245644e+09,2009-06-22-06:10,hadoop0144,1195430400,19-Nov-07
203,hadoop-release-0.15.0.tar.gz,Hadoop_0150,1.245644e+09,2009-06-22-06:10,hadoop0150,1194048000,3-Nov-07
204,hadoop-release-0.15.1.tar.gz,Hadoop_0151,1.245644e+09,2009-06-22-06:10,hadoop0151,1196294400,29-Nov-07
205,hadoop-release-0.15.2.tar.gz,Hadoop_0152,1.245644e+09,2009-06-22-06:10,hadoop0152,1198800000,28-Dec-07
206,hadoop-release-0.15.3.tar.gz,Hadoop_0153,1.245644e+09,2009-06-22-06:10,hadoop0153,1200441600,16-Jan-08
207,hadoop-release-0.16.0.tar.gz,Hadoop_0160,1.245644e+09,2009-06-22-06:10,hadoop0160,1202083200,4-Feb-08


In [12]:
net_csv1 = [i for i in csv_files if  i.find('sys') !=-1]

In [13]:
# net_csv
for net in net_csv1:
    print(net)


D:/mm_thesis/Software_graphs/D_graph/D_sys/


In [15]:
net_csv1

['D:/mm_thesis/Software_graphs/D_graph/D_sys/']

In [16]:
net_csv_d = os.listdir(net_csv1[0])
print('files', net_csv_d)
net_csv =[os.path.join(net_csv1[0],net_csv_d[0])+'/' ]
print(net_csv)

files ['Jena']
['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/']


In [17]:
net_csv

['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/']

In [18]:
for net in net_csv:
#  for net in csv_files:
    net_list = os.listdir(net)
    net_files1 = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
    net_files =  [ i for i in net_files1 if (i.find('time')==-1)]
#     print('net_files  :',net_files)
    #print(net_files)
    for net_graph in net_files:
        print(net_graph)
        ver_lt = net_graph[:-1] #just remove / in the end
        ver_st = ver_lt.rfind('/')
        ver_no = ver_lt[ver_st+1:]
        print(ver_no)
#          Get the info from the lookup file 
#         print( Lookup_df[Lookup_df['netfile'] == ver_no])
        info_ver = Lookup_df[Lookup_df['netfile'] == ver_no]
        tar_name = Lookup_df[Lookup_df['netfile'] == ver_no]['tarfile'].values[0]
        t_stmp = Lookup_df[Lookup_df['netfile'] == ver_no]['stamp'].values[0]
      
            
        ver_end = tar_name.find('.tar')
        Version = tar_name[:ver_end]
        print(30*'___')
        print('to be save', Version,'-->', t_stmp)
        print(30*'___')
        nn_list = os.listdir(net_graph)
        #print(nn_list)
        c_path = [(os.path.join(net_graph,i )+"/")for i in nn_list if os.path.isdir(os.path.join(net_graph,i))]
#       
        print(30*'#')
        
#         print('c_path  :', c_path)
        for a_dir in c_path:
#             print(a_dir)
            adir_list1 = [ i for i in os.listdir(a_dir) if (i.find('fixed.csv') != -1 or i.find('edges') != -1)]
            adir_list = [i for i in adir_list1 if i.find('Ver_stmp') ==-1 ]
#             print('list1:  ',adir_list1)
#             print('list:      ', adir_list)
            for fl in adir_list:
#                 print(fl)
                save_name= "Ver_stmp_"+fl
                to_save = os.path.join(a_dir,save_name)

                f_read = a_dir+ fl 
#                 print('to read', f_read)
                df= pd.read_csv(f_read)
#                 df.insert(loc = 0, column = 'Version', value = Version)
#                 df.insert(loc = 1, column = 't_stmp', value = t_stmp)
                                                          
                df['Version'] = Version
                df['t_stmp'] = t_stmp
#                 print(df.head(2))
                print('to_save', to_save)
#                                                 
                df.to_csv(to_save,encoding='utf-8',index = False)
                                    

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/
jena-2.10.0
__________________________________________________________________________________________
to be save jena-jena-2.10.0 --> 1361318400
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2

to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.1/
jena-2.7.1
__________________________________________________________________________________________
to be save jena-jena-2.7.1 --> 1339545600
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.1/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.

to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.0/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.1.1/
jena-3.1.1
_______________________

to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.14.0/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/
jena-3.15.0
__________________________________________________________________________________________
to be save jena-jena-3.15.0 --> 1588636800
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.

to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.5.0/CollDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.5.0/CollDot_nodes/Ver_stmp_Nodes_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.5.0/InhDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.5.0/InhDot_csv/Ver_stmp_edges.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.5.0/InhDot_nodes/Ver_stmp_Nodes_fixed.csv
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.6.0/
jena-3.6.0
__________________________________________________________________________________________
to be save jena-jena-3.6.0 --> 1513123200
__________________________________________________________________________________________
##############################
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.6.0/CDot_csv/Ver_stmp_all_fixed.csv
to_save D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.6.0/C

--

# creating the txt files to be used in matlab, for other network metrics

In [19]:
from pathlib import Path
import os.path
import os
import numpy as np
import re
import pandas as pd
import shutil 
import networkx as nx
import dynetx as dn
import powerlaw
#from networkx.algorithms import community
import networkx.algorithms.community as nxcom
import community# as community_louvain
import matplotlib.pyplot as plt

In [31]:
csv_path =  'D:/mm_thesis/Software_graphs/D_graph/D_sys/'
softwareList = os.listdir(soft_path)
soft_files = [(os.path.join(soft_path,infile)+"/") for infile in softwareList]
csv_path_l = os.listdir(csv_path)
csv_files1 = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l if os.path.isdir(os.path.join(csv_path,infile))]


## to do the Hadoop part

In [37]:
csv_files1

['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/']

In [35]:
dot_file = ['collDot_time', 'InhDot_time' , 'CDot_time']

In [38]:
for i in csv_files1:
    print(i)
    for g in dot_file:
        print(g)
        g_path = os.path.join(i,g)+ '/'
        print(g_path)
        os.makedirs(g_path,exist_ok=True)


D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/
collDot_time
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/collDot_time/
InhDot_time
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/
CDot_time
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/


In [41]:
for i in csv_files1:
    print(i)

D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/


['D:/mm_thesis/Software_graphs/D_graph/all_metrics/',
 'D:/mm_thesis/Software_graphs/D_graph/D_matlab/',
 'D:/mm_thesis/Software_graphs/D_graph/D_sys/']

In [25]:
csv_net

NameError: name 'csv_net' is not defined

In [42]:

dot_types = ['CDot_', 'CollDot_', 'InhDot_']
# dot_types = ['CDot_csv', 'CollDot_csv', 'InhDot_csv']

file_name = 'Ver_stmp_edges.csv'
#for net in csv_net:
for net in csv_files1:
    print('net ----->', net)
    net_list = os.listdir(net)
    net_files = [(os.path.join(net,infile )+"/")for infile in net_list if os.path.isdir(os.path.join(net,infile))]
#     print(net_files)
    net_edges = [i for i in net_files if i.find('_time/') == -1]
#     print(net_edges)
    for d in dot_types:
        print('type:                      ', d)
        d_type = d+'csv'
        dot_path = [os.path.join(i,d_type) for i in net_edges]
#         print('dot path :                  ', dot_path)
        t_type = d + 'time'
        time_path = os.path.join(net,t_type)+'/'
        print('time_path              ', time_path)
        print(30*'+')
        icount = 0
        for dfile in dot_path:
            icount = icount + 1
            edge_file  =  os.path.join(dfile,file_name)
            print('edge file:                     ' , edge_file)
            edge_df = pd.read_csv(edge_file)
            e = edge_df[edge_df[['source', 'target']].duplicated() == False]
            x_t = edge_df['t_stmp'].tolist()[0]
            print('x_t=====================', x_t)
            gt = nx.from_pandas_edgelist(e, source = 'source', target='target',edge_attr=True, create_using=nx.DiGraph())
        
            x_t = str(x_t)
            x_name = x_t+'.txt'
            print('time - path to make sure', time_path)
            x_file = os.path.join(time_path,x_name)
            print()
            tst =[e for e in gt.edges]
            print(len(tst))
            s_e =list( gt.edges)
            print(len(s_e))
            print('file to save x_file',x_file)
            print(50*'.')
            df = pd.DataFrame(s_e, columns =['source', 'target'])
            df.to_csv(x_file, sep='\t', index=False, header = False)
        print(' number of file :',icount)

            
    
    

net -----> D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/
type:                       CDot_
time_path               D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/
++++++++++++++++++++++++++++++
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CDot_csv\Ver_stmp_edges.csv
x_t===================== 1361318400
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/

30204
30204
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/1361318400.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.1/CDot_csv\Ver_stmp_edges.csv
x_t===================== 1368230400
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/

30871
30871
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/1368230400.txt
.................................................

x_t===================== 1588636800
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/

46673
46673
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/1588636800.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.16.0/CDot_csv\Ver_stmp_edges.csv
x_t===================== 1594252800
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/

47519
47519
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/1594252800.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.17.0/CDot_csv\Ver_stmp_edges.csv
x_t===================== 1604534400
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/

47671
47671
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_

x_t===================== 1344384000
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/

6920
6920
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/1344384000.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/CollDot_csv\Ver_stmp_edges.csv
x_t===================== 1350691200
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/

6861
6861
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/1350691200.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.0.0/CollDot_csv\Ver_stmp_edges.csv
x_t===================== 1437782400
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/

8824
8824
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_s

x_t===================== 1538092800
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/

10277
10277
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/1538092800.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-4.0.0/CollDot_csv\Ver_stmp_edges.csv
x_t===================== 1616803200
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/

9502
9502
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CollDot_time/1616803200.txt
..................................................
 number of file : 32
type:                       InhDot_
time_path               D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/
++++++++++++++++++++++++++++++
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/InhDot_csv\Ver_stmp_edges.csv
x_t================

..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.14.0/InhDot_csv\Ver_stmp_edges.csv
x_t===================== 1579132800
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/

4461
4461
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/1579132800.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.15.0/InhDot_csv\Ver_stmp_edges.csv
x_t===================== 1588636800
time - path to make sure D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/

4490
4490
file to save x_file D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/1588636800.txt
..................................................
edge file:                      D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.16.0/InhDot_csv\Ver_stmp_edges.csv
x_t===================== 

# deleted the Ver_stmp by hand for  this dir build it up again

In [34]:

	
df_1 = pd.DataFrame(
	[['Somu', 68, 84, 78, 96],
	['Kiku', 74, 56, 88, 85],
	['Ajit', 77, 73, 82, 87]],
	columns=['name', 'physics', 'chemistry','algebra','calculus'])

df_2 = pd.DataFrame(
	[['Amol', 72, 67, 91, 83],
	['Lini', 78, 69, 87, 92]],
	columns=['name', 'physics', 'chemistry','algebra','calculus'])	

df_1 = df_1.append(df_2, ignore_index=True)

In [35]:
df_1


,name,physics,chemistry,algebra,calculus
0,Somu,68,84,78,96
1,Kiku,74,56,88,85
2,Ajit,77,73,82,87
3,Amol,72,67,91,83
4,Lini,78,69,87,92
